# Little Lemon analysis and sales report

You have gained all the necessary skills and knowledge required to interact with the back-end MySQL database from your Python-based front-end application via the standard database client MySQL Connector/Python. 

Follow the instructions in the exercises given below and create a Jupyter notebook to produce a Little Lemon analysis and sales report. You need to submit this report for your peers to review.

In [77]:
# run in kernel python 3.8.10 64-bit
# Needed libraries for this exercise:

!pip install mysql-connector-python

import mysql.connector as connector

from mysql.connector.pooling import MySQLConnectionPool

import datetime as dt

## Task 1: Stablish a connection

Complete the following steps to establish a connection pool:

* To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.
* To find the information on the error, import the Error class from MySQL Connector/Python.
*  Define your database configurations as a Python dictionary object called dbconfig.
* Establish a connection pool [pool_name = pool_b] with two connections. 
* Implement error handling using a try-except block in case the connection fails. 

In [78]:
dbconfig={"database":"little_lemon_db", "user":"root", "password":""}

In [80]:
## here yoyr code
try:
    pool = MySQLConnectionPool(
        pool_name="pool_b",
        pool_size=2,
        #database='little_lemon',
        #user='root',
        #password=''
        **dbconfig
        )
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is: ",pool.pool_size)
        #
except Error as er:
    print("error code", er.errno)
    print("error message", er.msg)

The connection pool is created with a name:  pool_b
The pool size is:  2


## Task 2: 

Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:

TIP: You need to add a connection to connect the third guest.


Guest 1:

    Table Number: 8
    First Name: Anees
    Last Name: Java
    Booking Time: 18:00
    EmployeeID: 6

Guest 2:

    Table Number: 5
    First Name: Bald
    Last Name: Vin
    Booking Time: 19:00
    EmployeeID: 6

Guest 3:

    Table Number: 12
    First Name: Jay
    Last Name: Kon
    Booking Time: 19:30 
    EmployeeID: 6

Return all the connections back to the pool. 

_TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message._ 

In [81]:
## 
guests = {'g1':{'TableNo':8, 'GuestFirstName':'Anees','GuestLastName':'Java', 'BookingTime':'18:00','EmployeeID':6},
'g2':{'TableNo':5, 'GuestFirstName':'Bald','GuestLastName':'Vin', 'BookingTime':'19:00','EmployeeID':6},
'g3':{'TableNo':12, 'GuestFirstName':'Jay','GuestLastName':'Kon', 'BookingTime':'19:30','EmployeeID':6}}



In [82]:
sql_guest_insert_rows = """
INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES (%(tableno)s, %(guestfirstname)s, %(guestlastname)s, %(bookingtime)s, %(employeeid)s ) ;
"""

In [83]:
# iterate through dictionary of Guests

for  key,value in guests.items():
    print("======== Connecting for guest {} ".format(key))
    print(value)
    tableno=value['TableNo']
    guestfirstname=value['GuestFirstName']
    guestlastname=value['GuestLastName']
    bookingtime=value['BookingTime']
    employeeid=value['EmployeeID']
    try:
        conn=pool.get_connection()
    except:
        print("----full pools! adding new connections for {}: ".format(key))
        pool.add_connection()
        conn=pool.get_connection()
    ### now we have connection for sure!
    if conn.is_connected():
        cursor=conn.cursor()
        cursor.execute(sql_guest_insert_rows, {'tableno':tableno,'guestfirstname':guestfirstname , 'guestlastname': guestlastname,'bookingtime':bookingtime,'employeeid':employeeid})
    else:
        print("no live connection made. Maybe server is down")
    # now that you inset changes you have to commit them in the database
    print("Sending results to DB")
    conn.commit()
    print("Returning connection to the pool!")
    # now give back the connection to the pool!
    conn.close()

======== Connecting for guest g1 
{'TableNo': 8, 'GuestFirstName': 'Anees', 'GuestLastName': 'Java', 'BookingTime': '18:00', 'EmployeeID': 6}
Sending results to DB
Returning connection to the pool!
======== Connecting for guest g2 
{'TableNo': 5, 'GuestFirstName': 'Bald', 'GuestLastName': 'Vin', 'BookingTime': '19:00', 'EmployeeID': 6}
Sending results to DB
Returning connection to the pool!
======== Connecting for guest g3 
{'TableNo': 12, 'GuestFirstName': 'Jay', 'GuestLastName': 'Kon', 'BookingTime': '19:30', 'EmployeeID': 6}
Sending results to DB
Returning connection to the pool!


In [84]:
### check current content in the database. 

sql_query_insert_info = """SELECT * FROM Bookings;"""

conn=pool.get_connection()
#
if conn.is_connected():
    cursor=conn.cursor()
    cursor.execute(sql_query_insert_info)
    results_test = cursor.fetchall()
    print(cursor.column_names)
    for row in results_test:
        print(row)
else:
    print("no live connection made")
print("Returning connection to the pool")
conn.close()

('BookingID', 'TableNo', 'GuestFirstName', 'GuestLastName', 'BookingSlot', 'EmployeeID')
(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1)
(2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1)
(3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3)
(4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4)
(5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2)
(6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)
(7, 8, 'Anees', 'Java', datetime.timedelta(seconds=64800), 6)
(8, 5, 'Bald', 'Vin', datetime.timedelta(seconds=68400), 6)
(9, 12, 'Jay', 'Kon', datetime.timedelta(seconds=70200), 6)
Returning connection to the pool


## Task 3: 

Create a report containing the following information:

1.  The name and EmployeeID of the Little Lemon manager
2.  The name and role of the employee who receives the highest salary.
3.  The number of guests booked between 18:00 and 20:00.
4.  The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [85]:
## STEP 1: sql statements

sql_report_1 ="""
SELECT EmployeeID,
    Name,
    Role
    FROM Employees
    WHERE Role ='Manager';
"""
sql_report_2 ="""
SELECT EmployeeID,
    Name,
    Role,
    CAST(REPLACE(REPLACE(Annual_Salary,"$",""),",","") AS UNSIGNED INTEGER) AS annualsalary
FROM Employees
ORDER BY annualsalary DESC
LIMIT 1;
"""

sql_report_3 ="""
SELECT BookingID,
    HOUR(BookingSlot) as hourbooking
FROM Bookings
WHERE HOUR(BookingSlot) >=18 
    AND  HOUR(BookingSlot) <=20 
;
"""

sql_report_4 ="""
SELECT  BookingID,
    CONCAT(GuestFirstName,' ',GuestLastName) as fullname
    FROM Bookings
    ORDER BY BookingSlot ;
"""





In [89]:
# ensure that there is at least one connection available by closing a open connection.
try:
    if conn.is_connected():
        conn.close()
except:
    print("There are connections available")
    

There are connections available


In [90]:
### performs all the calls to the sqls in order and return the values into a variable before print all the results in a report style:

conn=pool.get_connection()
#
if conn.is_connected():
    cursor=conn.cursor(buffered=True)
    cursor.execute(sql_report_1)
    #1.  The name and EmployeeID of the Little Lemon manager
    #2.  The name and role of the employee who receives the highest salary.
    #3.  The number of guests booked between 18:00 and 20:00.
    #4.  The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.
    print("================================================================================") 
    print("=================================== REPORT =====================================")
    print("================================================================================\n") 
    print("-----------------------------------------------------------------------------------")
    print("1.  The name and EmployeeID of the Little Lemon manager\n")
    print(cursor.column_names)
    results_1 = cursor.fetchall()
    for r1 in results_1:
        print(r1) #cursor.column_names)
    print("-----------------------------------------------------------------------------------")
    #for row in results_test:
    #    print(row)
    cursor.execute(sql_report_2)
    print("-----------------------------------------------------------------------------------")
    print("2.  The name and role of the employee who receives the highest salary.\n")
    print(cursor.column_names)
    results_2 = cursor.fetchall()
    for r2 in results_2:
        print(r2)
    print("-----------------------------------------------------------------------------------")
    #
    cursor.execute(sql_report_3)
    print("-----------------------------------------------------------------------------------")
    print("3.  The number of guests booked between 18:00 and 20:00.\n")
    print(cursor.column_names)
    results_3 = cursor.fetchall()
    for r3 in results_3:
        print(r3)
    print("-----------------------------------------------------------------------------------")
    #
    cursor.execute(sql_report_4)
    print("-----------------------------------------------------------------------------------")
    print("4.  The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.\n")
    print(cursor.column_names)
    results_4 = cursor.fetchall()
    for r4 in results_4:
        print(r4)
    print("-----------------------------------------------------------------------------------")
    #
    print("================================================= END OF REPORT ===========================================")
    # 

else:
    print("no live connection made")
print("Returning connection to the pool , to male it available for other users.")
conn.close()


=================================== REPORT =====================================

-----------------------------------------------------------------------------------
1.  The name and EmployeeID of the Little Lemon manager

('EmployeeID', 'Name', 'Role')
(1, 'Mario Gollini', 'Manager')
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
2.  The name and role of the employee who receives the highest salary.

('EmployeeID', 'Name', 'Role', 'annualsalary')
(1, 'Mario Gollini', 'Manager', 70000)
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
3.  The number of guests booked between 18:00 and 20:00.

('BookingID', 'hourbooking')
(1, 19)
(2, 19)
(5, 18)
(6, 20)
(7, 18)
(8, 19)
(9, 19)
-----------------------------------------------------------------------------

In [91]:
# ensure that there is at least one connection available by closing a open connection:
try:
    if conn.is_connected():
        conn.close()
except:
    print("There are connections available")

There are connections available


## TASK 4:

Create a stored procedure named BasicSalesReport that returns the following statistics: 

* Total sales
* Average sale
* Minimum bill paid
* Maximum bill paid

In [92]:
sql_procedure_basicsales = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT SUM(BillAmount) AS 'Total sales',
        AVG(BillAmount) AS 'Average sale',
        MIN(BillAmount) AS 'Minimum bill paid',
        MAX(BillAmount) AS 'Maximum bill paid'
    FROM Orders;
END
"""

In [93]:
# create and execute
# STEP 2
conn=pool.get_connection()
if conn.is_connected():
    cursor=conn.cursor()
    cursor.execute(sql_procedure_basicsales)
    print("procedure basicsales created")
else:
    print("no live connection made")
conn.close()
print("the connection is placed back into the pool for the next user to connect")



procedure basicsales created
the connection is placed back into the pool for the next user to connect


In [94]:
# STEP 3 : call procedure

conn=pool.get_connection()
if conn.is_connected():
    cursor=conn.cursor()
    cursor.callproc("BasicSalesReport")
    results = next(cursor.stored_results())
    columns = results.column_names
    print(columns)
    dataset = results.fetchall()

    for row in dataset:
        print(row)
else:
    print("no live connection made")
conn.close()
print("the connection is placed back into the pool for the next user to connect")

('Total sales', 'Average sale', 'Minimum bill paid', 'Maximum bill paid')
(Decimal('243'), Decimal('48.6000'), 37, 86)
the connection is placed back into the pool for the next user to connect


## TASK 5: 
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

* Get a connection from the pool.
* Create a buffered cursor.
* Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 
* Returned the connection back to the pool.

The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [95]:
## SQL test
sql_test_chefnotifications = """ 
SELECT 
    Bookings.BookingID,
    Bookings.BookingSlot,
    CONCAT(Bookings.GuestFirstName,' ',Bookings.GuestLastName) AS Guest_name,
    Employees.Name AS EmployeeName,
    Employees.Role
FROM Bookings
LEFT JOIN Employees
ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY BookingSlot ASC
LIMIT 3;
"""



In [96]:
# ensure there is at least one connection available by clossing an open connection:
try:
    if conn.is_connected():
        conn.close()
except:
    print("There are connections available")

There are connections available


In [97]:

## check
conn=pool.get_connection()
if conn.is_connected():
    cursor=conn.cursor(buffered=True)
    cursor.execute(sql_test_chefnotifications)
    notificationinfo=cursor.fetchall()
    #print(notificationinfo)
    #print("procedure basicsales created")
    for notification in notificationinfo:
        print("-----------------------\n Booking slot {} .\n Guest name: {} .\n Assigned to {} : ({}) .\n-----------------------".format(notification[1] ,notification[2], notification[3], notification[4] ))
else:
    print("No live connection made")
conn.close()
print("\n\nThe connection is placed back into the pool for the next user to connect.")

-----------------------
 Booking slot 15:00:00 .
 Guest name: Vanessa McCarthy .
 Assigned to Giorgos Dioudis : (Head Chef) .
-----------------------
-----------------------
 Booking slot 17:30:00 .
 Guest name: Marcos Romero .
 Assigned to Fatma Kaya : (Assistant Chef) .
-----------------------
-----------------------
 Booking slot 18:00:00 .
 Guest name: Anees Java .
 Assigned to John Millar : (Receptionist) .
-----------------------


The connection is placed back into the pool for the next user to connect.
